In [1]:
import numpy as np
import pandas as pd
import altair as alt
import datetime as dt
import plotly.io as pio
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%matplotlib inline

pio.renderers.default = "plotly_mimetype+notebook_connected"
# pyo.init_notebook_mode()
# import plotly.io as pio
#pio.renderers.default = 'iframe_connected'

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last"

## Plotly with dropdown menu and slider
dropdown menu as the households, and slider as the date (first 2 weeks of the data)

In [3]:

## import data (processed by R)

## ten-minus summary for each household each day smart meter data
ten_min_day = pd.read_csv('../data/ten_min_day.csv')

## formatting for later use
ten_min_day['datetime'] = ten_min_day["date"] +"-"+ ten_min_day["time"]
ten_min_day['date']= pd.to_datetime(ten_min_day['date'])
ten_min_day['datetime']= pd.to_datetime(ten_min_day['datetime'])
ten_min_day.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24192 entries, 0 to 24191
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       24192 non-null  object        
 1   date       24192 non-null  datetime64[ns]
 2   value      24192 non-null  float64       
 3   household  24192 non-null  int64         
 4   phase      24192 non-null  int64         
 5   datetime   24192 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 1.1+ MB


In [4]:

## add a column as daily total consumption
#daily_tol = ten_min_day.groupby(['date','household']).agg({'value': 'sum'})['value']
#ten_min_day = pd.merge(ten_min_day, daily_tol,  how='left', left_on=['date','household'], right_on = ['date','household'])
#ten_min_day = ten_min_day.rename(columns={"value_x": "ten_mins", "value_y": "daily_tol"})
ten_min_day.head()

,time,date,value,household,phase,datetime
0,00:00,2012-06-27,0.012044,4,3,2012-06-27 00:00:00
1,00:10,2012-06-27,0.011858,4,3,2012-06-27 00:10:00
2,00:20,2012-06-27,0.011640,4,3,2012-06-27 00:20:00
3,00:30,2012-06-27,0.011608,4,3,2012-06-27 00:30:00
4,00:40,2012-06-27,0.011880,4,3,2012-06-27 00:40:00


In [368]:

# INITIALIZE GRAPH OBJECT
fig = go.Figure(layout=dict(
    legend=dict(groupclick="toggleitem")))

# fig = make_subplots(rows=4, cols=1, vertical_spacing=0.065, shared_xaxes=True)

# ADD TRACES: [c1*y1, c1*y2, c1*y3,..c2*y1, c2*y2,..]
for house in pd.unique(ten_min_day.household):  # 3 household
    for s in pd.unique(ten_min_day.date):  # 14 days

        df1 = ten_min_day.loc[(ten_min_day['household'] == house) & (
            ten_min_day['date'] == s)]
        total_df = df1.loc[(df1['phase'] == 0)]
        p1_df = df1.loc[(df1['phase'] == 1)]
        p2_df = df1.loc[(df1['phase'] == 2)]
        p3_df = df1.loc[(df1['phase'] == 3)]

        fig.add_trace(
            go.Scatter(x=list(total_df.time), y=list(round(total_df.value,3)), visible=False, name="Total",
                       line=dict(color='red'),))
        fig.add_trace(
            go.Scatter(x=list(p1_df.time), y=list(round(p1_df.value,3)), visible=False, name="Phase 1",
                       line=dict(color='blue')))
        fig.add_trace(
            go.Scatter(x=list(p2_df.time), y=list(round(p2_df.value,3)), visible=False, name="Phase 2",
                       line=dict(color='green')))
        fig.add_trace(
            go.Scatter(x=list(p3_df.time), y=list(round(p3_df.value,3)), visible=False, name="Phase 3",
                       line=dict(color='orange')))

        # total 4*14*3 traces

        # fig.add_trace(
        #     go.Scatter(
        #     visible=False,
        #     line=dict(color="#00CED1", width=6),
        #     name="Date : " + str(step),
        #     x=pd.unique(ten_min_day.date),
        #     y=ten_min_day.value))

# Create and add slider
steps = []

# for each year and each continent

for i in range(3*14):
    label = [str(s) for s in ten_min_day['date'].dt.strftime(
        '%Y-%m-%d(%a)').unique()]
    step = dict(
        method="update",
        label=label[i % 14],
        args=[{"visible": [False]*3*14*4},
        {"title": "Daily total consumption: " + str("%.2f" % ten_min_day.groupby(['date','household']).agg({'value': 'sum'})['value'][i]) + " kW.h"}],
        # layout attribute
    )

    step["args"][0]["visible"][i*4] = True
    step["args"][0]["visible"][i*4+1] = True
    step["args"][0]["visible"][i*4+2] = True
    step["args"][0]["visible"][i*4+3] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = []

for i in np.arange(0, 3*14, 14):
    slider = [dict(
        active=0,
        currentvalue={"prefix": "Date: " },
        pad={"t": 50},
        steps=steps[i:i+14],


    )]
    sliders.append(slider)

# INITIALIZE
# MAKE FIRST TRACE VISABLE
fig.data[0].visible = True
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True
fig.update_layout(sliders=sliders[0])


# ADD DROPDOWN TO CHANGE TYPE
# "restyle": modify data or data attributes
# "relayout": modify layout attributes
# "update": modify data and layout attributes
# "animate": start or pause an animation

fig.update_layout(
    title=dict(text="Smart power 10-mins usage by household",
               font=dict(size=25), yref='paper'),
    legend_title="Phase",
    xaxis_title="Time",
    yaxis_title="Energy consumption, kW.h",
    plot_bgcolor="floralwhite",
    showlegend=True,
    hovermode='x'

)
fig.update_xaxes(tickangle=-45,
                 showspikes=True,
                 spikemode='across',
                 spikesnap='cursor',
                 showline=True,
                 showgrid=True,
                 spikethickness=1,
                
                 )
# IMPORTANT WITH MULTIPLE ARGS YOU NEED EACH IN A DICTIONARY FOR SOME REASON
fig.update_layout(

    updatemenus=[
        dict(
            buttons=list([
                dict(
                    label="Household 04",
                    method="update",
                    args=[{"visible": [False]*0*4+[True]*4+[False]
                           * (3*14-1)*4}, {"sliders": sliders[0]}],


                ),
                dict(
                    label="Household 05",
                    method="update",
                    args=[{"visible": [False]*14*4+[True]*4+[False]
                           * (3*14-1-14)*4}, {"sliders": sliders[1]}],
                ),
                dict(
                    label="Household 06",
                    method="update",
                    args=[{"visible": [False]*28*4+[True]*4+[False]
                           * (3*14-1-14*2)*4}, {"sliders": sliders[2]}],

                )
            ]),
            direction="down",
            showactive=True,
            pad={"r": 10, "t": 10},
            x=0.89,
            xanchor="left",
            y=1.2,
            yanchor="auto",
        
        )
    ]
)

fig.show()
# import plotly as plt
#fig.show(renderer="notebook_connected")
# # Plot it and save as basic-line.html
# plt.offline.plot(fig, filename = 'Test-1.html', auto_open = False)


# # Show HTML
# from IPython.display import HTML
# HTML(filename='Test-1.html')

## Altair

In [21]:

## import data
## average hourly summary from all existing date by household for each applicance

hourly_app = pd.read_csv('../data/daily_consum.csv')
hourly_app.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   column_label  552 non-null    int64  
 1   household     552 non-null    int64  
 2   daily_avg     552 non-null    float64
 3   appliance     552 non-null    int64  
 4   hour          552 non-null    int64  
 5   hour_avg      552 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 26.0 KB


In [79]:
hourly_app.head()

,column_label,household,daily_avg,appliance,hour,hour_avg,appliance_name
0,1,4,6.625066,1,1,0.026889,Fridge
1,1,4,6.625066,1,2,0.026279,Fridge
2,1,4,6.625066,1,3,0.024942,Fridge
3,1,4,6.625066,1,4,0.026053,Fridge
4,1,4,6.625066,1,5,0.026305,Fridge


In [30]:
alt.data_transformers.enable('default', max_rows=None)


DataTransformerRegistry.enable('default')

In [374]:
### first plot show the the average daily consumption 

selection_bar = alt.selection_single(fields=['household'],name='House')
color_bar = alt.condition(selection_bar,
                      alt.value('sienna'),
                      alt.value('lightgray'))

bar=(alt.Chart(hourly_app)
 .mark_bar(size=30)
 .encode(y=alt.Y('daily_avg:Q',scale=alt.Scale(domain=[0,7])),
         x='household:N',
         color=color_bar
        )
)

text_bar = bar.mark_text(
    align='center',
    baseline='bottom',
    dx=1  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text("daily_avg:Q", format=",.2f")
)


bar.title =["Daily average energy consumption","by household"]
bar.encoding.x.title = 'Households'
bar.encoding.y.title = 'Daily consumption, kW.h'
(bar + text_bar).properties(width=200).add_selection(selection_bar)


alt.LayerChart(...)

In [371]:
# pie chart to see the appliance use percentage

selection_app = alt.selection_single(encodings=['theta','color'],name='app')
selection_app1 = alt.selection_single(fields=['appliance_name'],name='app1')
# color_app = alt.condition(selection_app,
#                       'appliance_name',
#                       alt.value('lightgray'))
opacity_app = alt.condition(selection_app, alt.value(1.0), alt.value(0.2))

base = alt.Chart(hourly_app).transform_filter(selection_bar
).encode(
    theta=alt.Theta("usage:Q", stack=True), 
    color=alt.Color('appliance_name:N', legend=None),
    opacity=opacity_app,
    tooltip='combined_tooltip:N',
).transform_aggregate(
    usage = 'sum(hour_avg)',
    groupby=["appliance_name"]
    
).transform_calculate(
    round_usage = 'round(datum.usage*1000)/1000',
    combined_tooltip = "'Usage: ' + datum.round_usage + ' kW.h'"
    )

pie = base.mark_arc(outerRadius=120)
text_pie = base.mark_text(baseline="bottom",radius=120, size=12,lineBreak=" ",radiusOffset=22
            ).encode(text="appliance_name:N"
            )

# pie plot title
pie.title =["Usage by appliances"]

(bar + text_bar).properties(width=200,height=310).add_selection(selection_bar) | (pie + text_pie).properties(height=310).add_selection(selection_app).add_selection(selection_app1)


alt.HConcatChart(...)

In [369]:
## barplot show usage for every hour

color_hour = alt.condition(selection_app,
                      alt.Color('appliance_name:N'),
#                       alt.value('steelblue'),
                      alt.value('lightgray'))

bar_hour=(alt.Chart(hourly_app)
 .mark_bar()
 .transform_filter(selection_bar
 ).transform_filter(selection_app1)
 ).encode(x=alt.X('hour:N'),
         y=alt.Y('hour_avg:Q'),
         color=alt.Color('appliance_name:N', legend=None)
        
        )


bar_hour.title ="Appliance energy consumption"
bar_hour.encoding.x.title = 'Hour'
bar_hour.encoding.y.title = 'Energy consumption, kW.h'

(bar + text_bar).properties(width=200,height=310).add_selection(selection_bar) | (pie + text_pie).properties(height=310).add_selection(selection_app).add_selection(selection_app1) | bar_hour.properties(height=310)


/Users/xiaojingni/miniforge3/envs/anly503/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



alt.HConcatChart(...)